# Parte 6 - TOPSIS pra seleção de hinos similares

Seleção de hinos similares a partir de um hino de referência, utilizando o método TOPSIS, considerando as matrizes de similaridade, categorias, clusterings, e tópicos.

In [ ]:
import pandas as pd

hinos_analise: pd.DataFrame = pd.read_pickle(
    "../assets/hinos_analise_embeddings_complete.pkl"
)
similarity_matrix_words: pd.DataFrame = pd.read_pickle(
    "../assets/similarity_matrix_word_embeddings_tfidf.pkl"
)
similarity_matrix_sent: pd.DataFrame = pd.read_pickle(
    "../assets/similarity_matrix_sentence_embeddings.pkl"
)

A matriz de decisão é construída com relação a um hino de referência, e as alternativas são os demais hinos.

In [ ]:
hino_sample = hinos_analise.sample().iloc[0]
hinos_restantes = hinos_analise[hinos_analise.index != hino_sample.name].copy()
# se categoria_id é a mesma de sample, então 1, senão 0
hinos_restantes["categoria_id"] = (
    hinos_restantes["categoria_id"] == hino_sample["categoria_id"]
).astype(int)
hinos_restantes["word_cluster"] = (
    hinos_restantes["word_cluster"] == hino_sample["word_cluster"]
).astype(int)
hinos_restantes["NMF_topic"] = (
    hinos_restantes["NMF_topic"] == hino_sample["NMF_topic"]
).astype(int)
hinos_restantes["sent_cluster"] = (
    hinos_restantes["sent_cluster"] == hino_sample["sent_cluster"]
).astype(int)
hinos_restantes["BERT_topic"] = (
    hinos_restantes["BERT_topic"] == hino_sample["BERT_topic"]
).astype(int)
hinos_restantes = hinos_restantes[
    [
        "nome",
        "texto_limpo",
        "categoria_id",
        "word_cluster",
        "NMF_topic",
        "sent_cluster",
        "BERT_topic",
    ]
]
hinos_restantes

In [ ]:
similarity_matrix_words_sample = similarity_matrix_words.loc[
    hino_sample.name, hinos_restantes.index
]
similarity_matrix_sent_sample = similarity_matrix_sent.loc[
    hino_sample.name, hinos_restantes.index
]
hinos_restantes["sim_word"] = similarity_matrix_words_sample.values
hinos_restantes["sim_sent"] = similarity_matrix_sent_sample.values
hinos_restantes

In [ ]:
categories = ["categoria_id", "word_cluster", "NMF_topic", "sent_cluster", "BERT_topic", "sim_word", "sim_sent"]
weights = [1, 2, 4, 3, 5, 6, 7]
weights = [w / sum(weights) for w in weights]
profit_cost = [1, 1, 1, 1, 1, 1, 1]

In [ ]:
from topsis import TOPSIS

output = TOPSIS(
    hinos_restantes[categories],
    weights,
    profit_cost,
)
scores = pd.DataFrame(output[1])
normalized_weighted_decision_matrix = output[0]
result = pd.concat([normalized_weighted_decision_matrix, scores], axis=1)
result.columns = categories + ["score"]
# result = result.sort_values(by="score", ascending=False)


In [ ]:
print("Sugestões para o hino:", hino_sample.name, hino_sample["nome"])
hinos_restantes["score"] = result["score"].values
hinos_restantes = hinos_restantes.sort_values(by="score", ascending=False)
hinos_restantes[["nome", "texto_limpo", "score"]].head(10)